In [1]:
import seaborn as sns 
data = sns.load_dataset('tips')
data.head()


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [2]:
data['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [3]:
data['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [4]:
data.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
encoder = LabelEncoder()

In [7]:
# Encoding wrt time 
data['time'] = encoder.fit_transform(data['time'])

In [8]:
X = data.drop(labels = ['time'], axis = 1) 
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [9]:
y = data.time
y.head()

0    0
1    0
2    0
3    0
4    0
Name: time, dtype: int64

In [10]:
X['day'].value_counts()

Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split 

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20, random_state = 42)

In [13]:
X_train

,total_bill,tip,sex,smoker,day,size
228,13.28,2.72,Male,No,Sat,2
208,24.27,2.03,Male,Yes,Sat,2
96,27.28,4.00,Male,Yes,Fri,2
167,31.71,4.50,Male,No,Sun,4
84,15.98,2.03,Male,No,Thur,2
...,...,...,...,...,...,...
106,20.49,4.06,Male,Yes,Sat,2
14,14.83,3.02,Female,No,Sun,2
92,5.75,1.00,Female,Yes,Fri,2
179,34.63,3.55,Male,Yes,Sun,2


In [14]:
from sklearn.pipeline import Pipeline 
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 

In [15]:
cat_col = ['sex', 'smoker', 'day']

num_col = ['total_bill', 'tip', 'size']

In [16]:
num_pipeline = Pipeline(
    steps = [
       ('imputer',  SimpleImputer(strategy = 'median')), 
       ('scaler', StandardScaler())
    ]
)

cat_pipeline = Pipeline(
    
    steps = [
        ( 'imputer', SimpleImputer(strategy = 'most_frequent')),
        ('encoder', OneHotEncoder())
    ]
)
                                  

In [17]:
preprocessor = ColumnTransformer([
    
    ('num_pipeline', num_pipeline, num_col), 
    ('cat_pipeline', cat_pipeline, cat_col)
])


In [18]:
X_train = preprocessor.fit_transform(X_train) 
X_test = preprocessor.transform(X_test)

In [19]:
X_train

array([[-0.79306155, -0.2580329 , -0.61214068, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.46322744, -0.74211442, -0.61214068, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.80730659,  0.6399734 , -0.61214068, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.65383098, -1.46472887, -0.61214068, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.64749986,  0.32426806, -0.61214068, ...,  0.        ,
         1.        ,  0.        ],
       [ 2.75289699, -0.41237773,  0.45363997, ...,  1.        ,
         0.        ,  0.        ]])

In [20]:
y_train

228    0
208    0
96     0
167    0
84     1
      ..
106    0
14     0
92     0
179    0
102    0
Name: time, Length: 195, dtype: int64

In [21]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier

In [22]:
models = {
    'random_forest': RandomForestClassifier(), 
    'logistic_regression' : LogisticRegression(), 
    'decision_tree' : DecisionTreeClassifier()
}

In [23]:
from sklearn.metrics import accuracy_score
def evaluate_model(X_train, y_train, X_test, y_test, models): 
    report = {}
    for i in range(len(models)): 
        model = list(models.values())[i]
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred) 
        report[list(models.keys())[i]] =accuracy
    return report 

In [24]:
evaluate_model(X_train, y_train, X_test, y_test, models )

{'random_forest': 0.9591836734693877,
 'logistic_regression': 1.0,
 'decision_tree': 0.9387755102040817}

In [25]:
params = {
    'n_estimators': [50, 100, 200],
    'criterion': ['gini', 'entropy'], 
    'max_depth': [3, 5, 10]
} 

In [26]:
from sklearn.model_selection import RandomizedSearchCV

In [27]:
model = RandomForestClassifier()

In [28]:
model 

RandomForestClassifier()

In [29]:
CV = RandomizedSearchCV(model, param_distributions = params, scoring = 'accuracy', cv = 5, verbose = 3 )

In [30]:
CV.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=3, n_estimators=50;, score=0.974 total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=3, n_estimators=50;, score=0.974 total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=3, n_estimators=50;, score=0.974 total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=3, n_estimators=50;, score=0.923 total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=3, n_estimators=50;, score=0.923 total time=   0.1s
[CV 1/5] END criterion=entropy, max_depth=3, n_estimators=50;, score=0.974 total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=3, n_estimators=50;, score=0.949 total time=   0.1s
[CV 3/5] END criterion=entropy, max_depth=3, n_estimators=50;, score=0.974 total time=   0.1s
[CV 4/5] END criterion=entropy, max_depth=3, n_estimators=50;, score=0.923 total time=   0.1s
[CV 5/5] END criterion=entropy, max_depth=3, n_estimators=50;, score=0.923 total time=   0.1s
[CV 1/5] END c

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10],
                                        'n_estimators': [50, 100, 200]},
                   scoring='accuracy', verbose=3)

In [31]:
# Best Parameters 
CV.best_params_

{'n_estimators': 100, 'max_depth': 10, 'criterion': 'entropy'}

In [32]:
best_model = RandomForestClassifier(n_estimators= 50, max_depth= 5, criterion= 'entropy')

In [33]:
best_model.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=5, n_estimators=50)

In [34]:
y_pred = best_model.predict(X_train)

In [35]:
y_pred 

array([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [36]:
best_accuracy = accuracy_score(y_train, y_pred)

In [37]:
print('Best Accuracy of the trainning data:' , best_accuracy*100) #Best accuracy of the Training Data 

Best Accuracy of the trainning data: 98.97435897435898


In [38]:
# Best accuracy of the Test Data 

y_pred = best_model.predict(X_test) 
best_accuracy = accuracy_score(y_test, y_pred)
print('Best Accuracy of the testing data:' , best_accuracy*100)

Best Accuracy of the testing data: 95.91836734693877
